### Boris Johnson - exploration

In [28]:
!pip install textblob

  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached tqdm-4.63.0-py2.py3-none-any.whl (76 kB)
  Using cached click-8.0.4-py3-none-any.whl (97 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.9/764.9 KB 7.6 MB/s eta 0:00:00a 0:00:01


In [35]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /home/alibor/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/alibor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alibor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alibor/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /home/alibor/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/alibor/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [33]:
import json
import pandas as pd
import os
from textblob import TextBlob
import nltk

In [17]:

print(os.getcwd())

/home/alibor/code/reichardtma/prestweets/notebooks


In [23]:
data = None
       
os.system(
        f'snscrape --jsonl --progress twitter-user borisjohnson > twitter-borisjohnson1.json')

tweets_df = pd.read_json('twitter-borisjohnson1.json', lines=True)
df = tweets_df[["id", "url", "date", "content",
                    "hashtags", "cashtags", "media", "lang"]]
if data is None:
    data = df
else:
    data = data.append(df)

df.content

Scraping, 100 results so far
Scraping, 200 results so far
Scraping, 300 results so far
Scraping, 400 results so far
Scraping, 500 results so far
Scraping, 600 results so far
Scraping, 700 results so far
Scraping, 800 results so far
Scraping, 900 results so far
Scraping, 1000 results so far
Scraping, 1100 results so far
Scraping, 1200 results so far
Scraping, 1300 results so far
Scraping, 1400 results so far
Scraping, 1500 results so far
Scraping, 1600 results so far
Scraping, 1700 results so far
Scraping, 1800 results so far
Scraping, 1900 results so far
Scraping, 2000 results so far
Scraping, 2100 results so far
Scraping, 2200 results so far
Scraping, 2300 results so far
Scraping, 2400 results so far
Scraping, 2500 results so far
Scraping, 2600 results so far
Scraping, 2700 results so far
Scraping, 2800 results so far
Scraping, 2900 results so far
Scraping, 3000 results so far
Scraping, 3100 results so far
Scraping, 3200 results so far
Scraping, 3300 results so far
Scraping, 3400 resu

0       Ahead of the NATO and G7 meetings this week, I...
1       I spoke to @KlausIohannis earlier and welcomed...
2       This World Down Syndrome Day, I’d like to than...
3       Terrible news about the China Eastern Airlines...
4       I spoke to President @ZelenskyyUa this afterno...
                              ...                        
4318    Again Mili says he would make cuts - on what a...
4319    How is Mili going to cut the deficit? He can't...
4320    Good to see yet more business leaders backing ...
4321    Cracking morning in Hendon with a great swathe...
4322    Good evening Twittersphere. This is Boris, com...
Name: content, Length: 4323, dtype: object

In [25]:
boris = df

In [31]:
boris.content.describe

<bound method NDFrame.describe of 0       Ahead of the NATO and G7 meetings this week, I...
1       I spoke to @KlausIohannis earlier and welcomed...
2       This World Down Syndrome Day, I’d like to than...
3       Terrible news about the China Eastern Airlines...
4       I spoke to President @ZelenskyyUa this afterno...
                              ...                        
4318    Again Mili says he would make cuts - on what a...
4319    How is Mili going to cut the deficit? He can't...
4320    Good to see yet more business leaders backing ...
4321    Cracking morning in Hendon with a great swathe...
4322    Good evening Twittersphere. This is Boris, com...
Name: content, Length: 4323, dtype: object>

In [38]:
blob = {}
for tweet in boris.content:
    each = TextBlob(tweet)
    blob[tweet] = each.sentiment.polarity

In [42]:
blob[boris.content[1]]

0.0